In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras
from tensorflow.compat.v2 import random as tfrandom
#for tf version > 2 use from tensorflow import random as tfrandom

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.utils import class_weight as sk_class_wgt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer as Normalizer
from sklearn.preprocessing import StandardScaler, QuantileTransformer,PowerTransformer,MinMaxScaler,MaxAbsScaler,RobustScaler
from sklearn.decomposition import TruncatedSVD
from datetime import datetime 
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
%matplotlib inline
plt.style.use('fivethirtyeight')

In [107]:
df01=pd.read_csv(r'C:\Users\tekin\Desktop\ML\Tweedy\TechCntVctUsnm_D1906.csv',index_col=False)
#print(dt_cl[:300])
df01.shape
#posset

(14390, 4)

In [108]:
SEED=7
np.random.seed(SEED)
tfrandom.set_seed(SEED)
keras.wrappers.scikit_learn.KerasClassifier.random_state=SEED
#help(grid)
print(keras.wrappers.scikit_learn.KerasClassifier.random_state)

7


In [109]:
y_vect=np.array(df01.edInput)
text_vect=np.array(df01.text[:].values)

In [110]:
X_train_raw, X_test_raw, y_train_all, y_test =train_test_split(text_vect,
            y_vect,random_state=SEED,test_size=0.1,stratify=y_vect)


In [111]:
#Use count vectorizer then tfid
#!!!!!!!!! we can add a column with a flag value that shows the tweet has pic or video into word vector

#lemma_vect =  TfidfVectorizer(tokenizer=None,token_pattern='\S+',
#                             min_df=4,lowercase=False,stop_words=None,ngram_range=(1,4),max_features=12000)
lemma_vect =  CountVectorizer(tokenizer=None,token_pattern='\S+',
                             min_df=1,lowercase=False,stop_words=None,ngram_range=(1,5),max_features=4200)
X_train_tmp=lemma_vect.fit_transform(X_train_raw)
X_test=lemma_vect.transform(X_test_raw)

In [112]:
X_train_tmp.shape

(12951, 4200)

https://www.kdnuggets.com/2019/04/normalization-vs-standardization-quantitative-analysis.html

Best classifier from each model:

    SVM + StandardScaler : 0.849
    MLP + PowerTransformer-Yeo-Johnson : 0.839
    KNN + MinMaxScaler : 0.813
    LR + QuantileTransformer-Uniform : 0.808
    NB + PowerTransformer-Yeo-Johnson : 0.752
    LDA + PowerTransformer-Yeo-Johnson : 0.747
    CART + QuantileTransformer-Uniform : 0.74
    RF + Normalizer : 0.723


In [ ]:
from scipy.sparse import csr_matrix 
sclPT=PowerTransformer()
#sclSTD=StandardScaler()
#normalizer=Normalizer()
#without normalizing
X_train_scl=sclPT.fit_transform(X_train_tmp.toarray())
#X_train_scl=X_train_tmp

X_test=sclPT.transform(X_test_tmp)
#X_test=X_test_tmp

In [ ]:
#svd= TruncatedSVD(n_components=Dim_num)


#X_train_scl = svd.fit_transform(X_train_scl)
#X_test = svd.transform(X_test_scl)
#print(svd.explained_variance_ratio_.sum())


In [ ]:
#this section is to print the names of the best features obtained from SVD
#check the names see if they are reasonable with your category

#feature_names=lemma_vect.get_feature_names()
#best_features = [feature_names[i] for i in svd.components_[0].argsort()[::-1]]
#print(best_features[:200])

In [113]:
X_train, X_val, y_train, y_val = train_test_split(X_train_tmp, y_train_all, 
                                                  test_size=0.2,stratify=y_train_all,random_state=SEED)

In [114]:
#WEIGHTED MODEL

negative = len(df01[df01['edInput'] == 0])
positive = len(df01[df01['edInput'] == 1])
print(negative,'\n',positive)
total = negative + positive
weight_for_0 = (1 / negative)*(total)/2.0 
weight_for_1 = (1 / positive)*(total)/2.0
#weight_for_1 = 100

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))
val_sample_weights = sk_class_wgt.compute_sample_weight(class_weight, y_val)
train_sample_weights = sk_class_wgt.compute_sample_weight(class_weight, y_train)
test_sample_weights = sk_class_wgt.compute_sample_weight(class_weight, y_test)
initial_bias = np.log([positive/total])
initial_bias

1595 
 12795
Weight for class 0: 4.51
Weight for class 1: 0.56


array([-0.11747905])

In [115]:
del X_train_raw
del X_train_tmp
del df01
del text_vect
#del X_train_scl
#del X_train_rd

def my_metric_fn(y_true, y_pred):
    sensitivity = keras.metrics.TrueNegatives(y_true, y_pred)/(keras.metrics.TrueNegatives(y_true, y_pred)+keras.metrics.FalsePositives(y_true, y_pred))
    return sensitivity 


In [128]:
keras.backend.clear_session()

In [129]:
EPOCHS = 200
BATCH_SIZE = 32
NEURONS=360
learn_rate=0.0001
krn_reg=regularizers.l2(0.01)
activation='relu'
act_reg=regularizers.l2(0.1)
dropout_rate=0.5


#model.compile(optimizer='adam', loss='mean_squared_error', metrics=[my_metric_fn])
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.SpecificityAtSensitivity(0.5,name='spcsens'),
      keras.metrics.SensitivityAtSpecificity(0.70,name='snsspec')    
]


In [130]:
class LearningRateDecay:
    def plot(self, epochs, title="Learning Rate Schedule"):
        # compute the set of learning rates for each corresponding
        # epoch
        lrs = [self(i) for i in epochs]
        # the learning rate schedule
        plt.style.use("ggplot")
        plt.figure()
        plt.plot(epochs, lrs)
        plt.title(title)
        plt.xlabel("Epoch #")
        plt.ylabel("Learning Rate")
        
class PolynomialDecay():
    def __init__(self, maxEpochs=100, initAlpha=0.01, power=1.0):
        # store the maximum number of epochs, base learning rate,
        # and power of the polynomial
        self.maxEpochs = maxEpochs
        self.initAlpha = initAlpha
        self.power = power
    def __call__(self, epoch):
        # compute the new learning rate based on polynomial decay
        decay = (1 - (epoch / float(self.maxEpochs))) ** self.power

        alpha = self.initAlpha * decay
        print("decay:",decay,">> lr:",alpha)
        # return the new learning rate
        return float(alpha)
schedule = PolynomialDecay(maxEpochs=EPOCHS, initAlpha=3e-4, power=1)

In [131]:
#schedule.__call__(90)

In [132]:
output_bias = keras.initializers.Constant(initial_bias)
def make_model(metrics = METRICS, output_bias=None):
    if output_bias is not None:
        output_bias = keras.initializers.Constant(output_bias)
    model = keras.Sequential([
          keras.layers.Dense(
              NEURONS, activation=activation,
              input_shape=(X_train.shape[-1],),kernel_regularizer=krn_reg
              ,activity_regularizer=act_reg),
          keras.layers.Dropout(dropout_rate),
          keras.layers.Dense(
              NEURONS/2, activation=activation,
              input_shape=(X_train.shape[-1],),kernel_regularizer=krn_reg
          ,activity_regularizer=act_reg),
          keras.layers.Dropout(dropout_rate),

          keras.layers.Dense(1, activation='sigmoid',
                         bias_initializer=output_bias),
  ])

#    model.compile(
#      optimizer=keras.optimizers.Adam(lr=1e-4),
#      loss=keras.losses.BinaryCrossentropy(),
#      metrics=metrics)

    return model

epoch=100
ep=lambda epoch: 1e-8 * 10**(epoch / 20)
float(ep(100))
#10**(1/20)

lr_schedule = keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))
optimizer = keras.optimizers.SGD(lr=1e-8, momentum=0.9)
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=METRICS)
history = model.fit(train_set, epochs=100, callbacks=[lr_schedule])

plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-5, 1e-3, 0, 30])

model = make_model() 
model.compile(
      optimizer=keras.optimizers.Adam(lr=1e-4),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=METRICS)
model.summary()

In [133]:
chkpnt_filepath=r"D:\work\SavedModels\Tweedy\Ckpt\chkpntwghts.{epoch:02d}-{val_weighted_accuracy:.4f}.hdf5" 
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_weighted_accuracy', 
    verbose=1,
    patience=80,
    mode='max',
    restore_best_weights=True)
model_checkpoint = keras.callbacks.ModelCheckpoint(
    filepath=chkpnt_filepath,
     save_best_only=True,  monitor="val_weighted_accuracy")
lrscheduler=keras.callbacks.LearningRateScheduler(schedule)

model = make_model()
model.summary()

model = make_model()
baseline_history = model.fit(
    X_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks = [early_stopping],
    validation_data=(X_val, y_val))


plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (12, 12)
plot_metrics(baseline_history)


def plot_cm(labels, predictions, p=0.5):
  cm = confusion_matrix(labels, predictions > p)
  plt.figure(figsize=(5,5))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix @{:.2f}'.format(p))
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  print('Legitimate Transactions Detected (True Negatives): ', cm[0][0])
  print('Legitimate Transactions Incorrectly Detected (False Positives): ', cm[0][1])
  print('Fraudulent Transactions Missed (False Negatives): ', cm[1][0])
  print('Fraudulent Transactions Detected (True Positives): ', cm[1][1])
  print('Total Fraudulent Transactions: ', np.sum(cm[1]))


train_predictions = model.predict(X_train, batch_size=BATCH_SIZE)
test_predictions= model.predict(X_test, batch_size=BATCH_SIZE)


results = model.evaluate(X_test, y_test,
                                           batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(model.metrics_names, results):
  print(name, ': ', value)
print()

plot_cm(y_test, test_predictions)


In [134]:
weighted_model = make_model(output_bias=initial_bias)
#weighted_model = make_model()
weighted_model.compile(
      optimizer=keras.optimizers.Adam(lr=1e-4),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=METRICS,weighted_metrics=[METRICS[4]])
weighted_model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 360)               1512360   
_________________________________________________________________
dropout (Dropout)            (None, 360)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 180)               64980     
_________________________________________________________________
dropout_1 (Dropout)          (None, 180)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 181       
Total params: 1,577,521
Trainable params: 1,577,521
Non-trainable params: 0
_________________________________________________________________


In [135]:
weighted_history = weighted_model.fit(
    X_train,
    y_train,
    sample_weight=train_sample_weights,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
   callbacks = [early_stopping,model_checkpoint,lrscheduler],
     validation_data=(X_val, y_val,val_sample_weights),
    # The class weights go here
    class_weight=class_weight) 

Train on 10360 samples, validate on 2591 samples
decay: 1.0 >> lr: 0.0003
Epoch 1/200
10360/10360 [==============================] - 7s 707us/sample - loss: 3.2234 - tp: 60.0000 - fp: 5.0000 - tn: 1144.0000 - fn: 9151.0000 - weighted_accuracy: 0.8861 - precision: 0.9231 - recall: 0.0065 - spcsens: 0.7285 - snsspec: 0.5539 - val_loss: 1.4907 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 287.0000 - val_fn: 2304.0000 - val_weighted_accuracy: 0.4998 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_spcsens: 0.9129 - val_snsspec: 0.8142
decay: 0.995 >> lr: 0.0002985
Epoch 2/200
10360/10360 [==============================] - 6s 608us/sample - loss: 1.1198 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 1149.0000 - fn: 9211.0000 - weighted_accuracy: 0.8892 - precision: 0.0000e+00 - recall: 0.0000e+00 - spcsens: 0.9669 - snsspec: 0.8720 - val_loss: 1.0709 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 287.0000 - val_fn: 2304.0000 - val_weighted_accuracy: 0.4998 - val_precision: 0.0

decay: 0.92 >> lr: 0.000276
Epoch 17/200
10360/10360 [==============================] - 7s 634us/sample - loss: 0.5761 - tp: 6043.0000 - fp: 1.0000 - tn: 1148.0000 - fn: 3168.0000 - weighted_accuracy: 0.9611 - precision: 0.9998 - recall: 0.6561 - spcsens: 0.9991 - snsspec: 0.9492 - val_loss: 0.8649 - val_tp: 1420.0000 - val_fp: 41.0000 - val_tn: 246.0000 - val_fn: 884.0000 - val_weighted_accuracy: 0.7367 - val_precision: 0.9719 - val_recall: 0.6163 - val_spcsens: 0.9059 - val_snsspec: 0.8099
decay: 0.915 >> lr: 0.0002745
Epoch 18/200
10360/10360 [==============================] - 6s 624us/sample - loss: 0.5688 - tp: 6126.0000 - fp: 4.0000 - tn: 1145.0000 - fn: 3085.0000 - weighted_accuracy: 0.9598 - precision: 0.9993 - recall: 0.6651 - spcsens: 1.0000 - snsspec: 0.9480 - val_loss: 0.8349 - val_tp: 1495.0000 - val_fp: 46.0000 - val_tn: 241.0000 - val_fn: 809.0000 - val_weighted_accuracy: 0.7443 - val_precision: 0.9701 - val_recall: 0.6489 - val_spcsens: 0.9024 - val_snsspec: 0.8073
deca

10360/10360 [==============================] - 6s 607us/sample - loss: 0.4806 - tp: 6892.0000 - fp: 1.0000 - tn: 1148.0000 - fn: 2319.0000 - weighted_accuracy: 0.9713 - precision: 0.9999 - recall: 0.7482 - spcsens: 0.9991 - snsspec: 0.9735 - val_loss: 0.8098 - val_tp: 1716.0000 - val_fp: 75.0000 - val_tn: 212.0000 - val_fn: 588.0000 - val_weighted_accuracy: 0.7417 - val_precision: 0.9581 - val_recall: 0.7448 - val_spcsens: 0.9059 - val_snsspec: 0.7912
decay: 0.835 >> lr: 0.00025049999999999996
Epoch 34/200
10360/10360 [==============================] - 6s 619us/sample - loss: 0.4729 - tp: 6922.0000 - fp: 3.0000 - tn: 1146.0000 - fn: 2289.0000 - weighted_accuracy: 0.9702 - precision: 0.9996 - recall: 0.7515 - spcsens: 1.0000 - snsspec: 0.9777 - val_loss: 0.8285 - val_tp: 1628.0000 - val_fp: 59.0000 - val_tn: 228.0000 - val_fn: 676.0000 - val_weighted_accuracy: 0.7505 - val_precision: 0.9650 - val_recall: 0.7066 - val_spcsens: 0.9024 - val_snsspec: 0.7938
decay: 0.83 >> lr: 0.000249
Epoc

10360/10360 [==============================] - 6s 601us/sample - loss: 0.4171 - tp: 7378.0000 - fp: 4.0000 - tn: 1145.0000 - fn: 1833.0000 - weighted_accuracy: 0.9749 - precision: 0.9995 - recall: 0.8010 - spcsens: 1.0000 - snsspec: 0.9866 - val_loss: 0.8060 - val_tp: 1747.0000 - val_fp: 79.0000 - val_tn: 208.0000 - val_fn: 557.0000 - val_weighted_accuracy: 0.7415 - val_precision: 0.9567 - val_recall: 0.7582 - val_spcsens: 0.9094 - val_snsspec: 0.7865
decay: 0.755 >> lr: 0.00022649999999999998
Epoch 50/200
10360/10360 [==============================] - 6s 601us/sample - loss: 0.4252 - tp: 7280.0000 - fp: 4.0000 - tn: 1145.0000 - fn: 1931.0000 - weighted_accuracy: 0.9737 - precision: 0.9995 - recall: 0.7904 - spcsens: 0.9991 - snsspec: 0.9863 - val_loss: 0.8311 - val_tp: 1672.0000 - val_fp: 68.0000 - val_tn: 219.0000 - val_fn: 632.0000 - val_weighted_accuracy: 0.7444 - val_precision: 0.9609 - val_recall: 0.7257 - val_spcsens: 0.8990 - val_snsspec: 0.7834
decay: 0.75 >> lr: 0.000225
Epoc

10360/10360 [==============================] - 6s 606us/sample - loss: 0.3775 - tp: 7659.0000 - fp: 2.0000 - tn: 1147.0000 - fn: 1552.0000 - weighted_accuracy: 0.9798 - precision: 0.9997 - recall: 0.8315 - spcsens: 1.0000 - snsspec: 0.9937 - val_loss: 0.8064 - val_tp: 1815.0000 - val_fp: 86.0000 - val_tn: 201.0000 - val_fn: 489.0000 - val_weighted_accuracy: 0.7441 - val_precision: 0.9548 - val_recall: 0.7878 - val_spcsens: 0.9059 - val_snsspec: 0.7878
decay: 0.675 >> lr: 0.0002025
Epoch 66/200
10360/10360 [==============================] - 7s 637us/sample - loss: 0.3663 - tp: 7726.0000 - fp: 1.0000 - tn: 1148.0000 - fn: 1485.0000 - weighted_accuracy: 0.9814 - precision: 0.9999 - recall: 0.8388 - spcsens: 1.0000 - snsspec: 0.9940 - val_loss: 0.8159 - val_tp: 1806.0000 - val_fp: 88.0000 - val_tn: 199.0000 - val_fn: 498.0000 - val_weighted_accuracy: 0.7386 - val_precision: 0.9535 - val_recall: 0.7839 - val_spcsens: 0.9094 - val_snsspec: 0.7704- tn: 409.0000 - fn: 462.0000 - weighted_accu


KeyboardInterrupt: 

In [ ]:
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

#plot_graphs(weighted_history, "binacc")
plot_graphs(weighted_history, "weighted_accuracy")
plot_graphs(weighted_history, "loss")
plot_graphs(weighted_history, "precision")
plot_graphs(weighted_history, "spcsens")
plot_graphs(weighted_history, "snsspec")
plot_graphs(weighted_history, "fp")

In [ ]:
#train_predictions_weighted = weighted_model.predict(X_train, batch_size=BATCH_SIZE)
#test_predictions_weighted = weighted_model.predict(X_test, batch_size=BATCH_SIZE)

In [ ]:
weighted_results = weighted_model.evaluate(X_test, y_test,
                                           batch_size=BATCH_SIZE, verbose=0,sample_weight=test_sample_weights)
for name, value in zip(weighted_model.metrics_names, weighted_results):
  print(name, ': ', value)
print()
print("sensitivity:",weighted_results[3]/(weighted_results[2]+weighted_results[3]))
#plot_cm(y_test, test_predictions_weighted)


EPOCHS = 200;BATCH_SIZE = 32;NEURONS=240;act='linear'
loss :  0.9864012012057539
tp :  903.0
fp :  32.0
tn :  127.0
fn :  377.0
weighted_accuracy :  0.7520231
precision :  0.96577543
recall :  0.7054688
spcsens :  0.9056604
snsspec :  0.821875
weighted_accuracy :  0.7520231

sensitivity: 0.7987421


EPOCHS = 200;BATCH_SIZE = 32;NEURONS=280


loss :  0.9741253216619538
tp :  837.0
fp :  29.0
tn :  130.0
fn :  443.0
weighted_accuracy :  0.7356135
precision :  0.9665127
recall :  0.6539062
spcsens :  0.9056604
snsspec :  0.828125
weighted_accuracy :  0.7356135

sensitivity: 0.8176101


EPOCHS = 200;BATCH_SIZE = 32;NEURONS=240;act='relu'

loss :  0.8104747441015449
tp :  865.0
fp :  31.0
tn :  128.0
fn :  415.0
weighted_accuracy :  0.7402924
precision :  0.96540177
recall :  0.67578125
spcsens :  0.918239
snsspec :  0.8453125
weighted_accuracy :  0.7402924

sensitivity: 0.8050314

EPOCHS = 200;BATCH_SIZE = 32;NEURONS=240;act='relu'; val_size=0.2

loss :  0.8032975160388005
tp :  934.0
fp :  35.0
tn :  124.0
fn :  346.0
weighted_accuracy :  0.7547364
precision :  0.9638803
recall :  0.7296875
spcsens :  0.9119497
snsspec :  0.8390625
weighted_accuracy :  0.7547364

sensitivity: 0.7798742

1

EPOCHS = 200;BATCH_SIZE = 32;NEURONS=240;act='relu'; val_size=0.2

In [ ]:
print(weighted_results)

ld_model=keras.models.load_model(r"D:\work\SavedModels\Tweedy\Ckpt\chkpntwghts.158-0.7733.hdf5")
#chkpntwghts.44-0.7479.hdf5
ld_model_results = ld_model.evaluate(X_test, y_test,
                                           batch_size=BATCH_SIZE, verbose=0,sample_weight=test_sample_weights)
for name, value in zip(weighted_model.metrics_names, ld_model_results ):
  print(name, ': ', value)
print()



(954*.56+127*4.54)/((954*.56+127*4.54)+31*4.54+326*.56)


ct = datetime.now()
ct=('Model_'+str(ct.month).zfill(2)+str(ct.day).zfill(2)+'_'+str(ct.hour).zfill(2)+str(ct.minute).zfill(2))
print(ct)
weighted_model.save("D:\\work\\SavedModels\\Tweedy\\Techmodels\\"+ct)

https://github.com/tensorflow/tensorflow/blob/f45d6083b766183e045552dafa8e46586eb3d4fc/tensorflow/python/keras/metrics.py#L1500

    

@keras_export('keras.metrics.SensitivityAtSpecificity')
class SensitivityAtSpecificity(SensitivitySpecificityBase):
  """Computes best sensitivity where specificity is >= specified value.
  the sensitivity at a given specificity.
  `Sensitivity` measures the proportion of actual positives that are correctly
  identified as such (tp / (tp + fn)).
  `Specificity` measures the proportion of actual negatives that are correctly
  identified as such (tn / (tn + fp)).
  This metric creates four local variables, `true_positives`, `true_negatives`,
  `false_positives` and `false_negatives` that are used to compute the
  sensitivity at the given specificity. The threshold for the given specificity
  value is computed and used to evaluate the corresponding sensitivity.
  If `sample_weight` is `None`, weights default to 1.
  Use `sample_weight` of 0 to mask values.
  For additional information about specificity and sensitivity, see
  [the following](https://en.wikipedia.org/wiki/Sensitivity_and_specificity).
